In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Embedding, Dense, SimpleRNN, LSTM, Bidirectional, Flatten
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split

In [17]:
# 1 .데이터 로드
ids = movie_reviews.fileids()
reviews = [movie_reviews.raw(id) for id in ids]
categoris = [movie_reviews.categories(id)[0] for id in ids]
# 토큰화 및 정수 인코딩
max_word = 10000
tokenizer = Tokenizer(num_words=max_word, oov_token='UNK')
tokenizer.fit_on_texts(reviews)
# 텍스트를 시퀀스로 변환
x = tokenizer.texts_to_sequences(reviews)
# 시퀀스 패딩
maxlen = 500
x = pad_sequences(x,maxlen=maxlen,truncating='pre')
# 레이블 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categoris])
# 학습 테스트 데이터 분할
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y, random_state=42,test_size=0.2)

In [ ]:
# 2 .세가지 모델 구축 , 학습
# 시드고정(재현성)
np.random.seed(42)
tf.random.set_seed(42)
# 모델 성능
results = {}
# ------ model 1 baseline(Flatten + Dense)
model1 = Sequential([
    Embedding(max_word,32,input_length = maxlen),
    Flatten(),
    Dense(1,activation='sigmoid')
])
model1.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['acc'])
model1.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
score1 = model1.evaluate(x_test,y_test)
results['baseline'] = score1[1]
print(f'model1 정확도:{score1[1]:.4f}')

# ------ model 2 (SimpleRnn)
model2 = Sequential([
    Embedding(max_word,32,input_length = maxlen),
    SimpleRNN(32),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])
model2.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['acc'])
model2.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
score1 = model2.evaluate(x_test,y_test)
results['Simple_RNN'] = score1[1]
print(f'model2 정확도:{score1[1]:.4f}')



Epoch 1/10


c:\Users\playdata2\miniconda3\envs\LLM311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5234 - loss: 0.6926 - val_acc: 0.5469 - val_loss: 0.6883
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.9727 - loss: 0.5895 - val_acc: 0.5938 - val_loss: 0.6815
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.9812 - loss: 0.4401 - val_acc: 0.6313 - val_loss: 0.6652
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.9977 - loss: 0.2623 - val_acc: 0.6313 - val_loss: 0.6474
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.9992 - loss: 0.1362 - val_acc: 0.6250 - val_loss: 0.6339
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 1.0000 - loss: 0.0743 - val_acc: 0.6187 - val_loss: 0.6272
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 1.0000 - loss: 0.0449 - val_acc: 0.6250 - val_loss: 0.6245
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 1.0000 - loss: 0.0297 - val_acc: 0.6250 - val_loss: 0.6221
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 1.0000 - loss: 0.

In [ ]:
# 3 .성능 비교 및 시각화

In [ ]:
# 4 .예측 결과 분석

In [ ]:
# 5 .학습결과 해석